# Assignment 2

## General information

This assignment has two problems:
 - 1: Linear Discriminat Analysis
 - 2: Quadratic Discriminat Analysis

Utilize the designated cells within this notebook to complete the exercises. As for the Python exercises:
- Refrain from altering the provided code; simply fill in the missing portions as indicated.
- Do not use any additional libraries beyond those already included in the code (e.g., NumPy library).
- Make sure that the output of all code cells is visible in your submitted notebook. **The evaluator is NOT expected to execute your code before grading your submission.**
- Some problems include automatic checks (*assertions*) to verify basic aspects of your solution. However, passing these assertions does not guarantee that your solution is entirely correct. The final evaluation will always be determined by the evaluator.
- Avoid the use of for loops by using NumPy vectorized operations whenever possible.
   
Please identify the authors of this assignment in the cell below.

## Identification

* **Name: Liu Cong**
* **Student Number:**

* **Name: Ulloa Ferrer Leonardo**
* **Student Number:**

* **Name:**
* **Student Number:**
---

**Note:** This work is to be done in group of up to **3** elements. Use this notebook to answer all the questions. At the end of the work, you should **upload** the **notebook** and a **pdf file** with a printout of the notebook with all the results in the **moodle** platform.
To generate the pdf file we have first to covert the notebook to html using the command `!jupyter nbconvert --to html "ML_project2.ipynb"`, then open the html file and printout to PDF.

## Introduction

For this assignment, we will consider the task of predicting the quality of red wine (rated on an integer scale from 0 to 5) based on 11 chemical properties, such as pH, alcohol content, residual sugar, etc. This dataset was originally published in:

Cortez, Paulo, et al. ["Modeling wine preferences by data mining from physicochemical properties."](https://https://www.sciencedirect.com/science/article/abs/pii/S0167923609001377) Decision support systems 47.4 (2009): 547-553.

Since quality is a discrete and ordinal attribute, this problem could be framed as either a regression or classification task. For this assignment, we will treat it as a classification problem. Specifically, we have a **6-class classification dataset** ($y \in \{0, 1, \dots, 5\}$) where each **feature vector is 11-dimensional** ($\mathbf{x} \in \mathbb{R}^{11}$).

In the cell below, we load and prepare the data for you.

In [1]:
import pandas as pd

train_data = pd.read_csv("winequality_train.csv")
print("A training example:")
print(train_data.iloc[0])
print()
test_data = pd.read_csv("winequality_test.csv")
print("A test example:")
print(test_data.iloc[0])
print()

X_train = train_data.iloc[:, :-1].values
y_train = train_data.iloc[:, -1].values
X_test = test_data.iloc[:, :-1].values
y_test = test_data.iloc[:, -1].values
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

A training example:
fixed acidity            8.60000
volatile acidity         0.22000
citric acid              0.36000
residual sugar           1.90000
chlorides                0.06400
free sulfur dioxide     53.00000
total sulfur dioxide    77.00000
density                  0.99604
pH                       3.47000
sulphates                0.87000
alcohol                 11.00000
quality                  4.00000
Name: 0, dtype: float64

A test example:
fixed acidity            7.7000
volatile acidity         0.5600
citric acid              0.0800
residual sugar           2.5000
chlorides                0.1140
free sulfur dioxide     14.0000
total sulfur dioxide    46.0000
density                  0.9971
pH                       3.2400
sulphates                0.6600
alcohol                  9.6000
quality                  3.0000
Name: 0, dtype: float64

X_train shape: (1119, 11)
X_test shape: (480, 11)
y_train shape: (1119,)
y_test shape: (480,)


An important **preprocessing step** for multi-dimensional continuous data is to ensure all features have **zero mean** and **unit variance**. In the code below, we transform the data to achieve this.

**When normalizing the test data, we reuse the mean and standard deviation from the training data**, rather than recalculating them. This ensures a fair evaluation, since we want to classify new examples without assuming any prior knowledge about their distribution beyond what was learned during the training phase.

In [2]:
print("Training data statistics before normalization:")
print("  * mean:", X_train.mean(axis=0))
print("  * std:", X_train.std(axis=0))
print()
print("Test data statistics before normalization:")
print("  * mean:", X_test.mean(axis=0))
print("  * std:", X_test.std(axis=0))
print()

train_mean = X_train.mean(axis=0)
train_std = X_train.std(axis=0)
X_train = (X_train - train_mean) / train_std
X_test = (X_test - train_mean) / train_std

print("Training data statistics after normalization:")
print("  * mean:", X_train.mean(axis=0))
print("  * std:", X_train.std(axis=0))
print()
print("Test data statistics after normalization:")
print("  * mean:", X_test.mean(axis=0))
print("  * std:", X_test.std(axis=0))

Training data statistics before normalization:
  * mean: [ 8.30956211  0.53313226  0.27025022  2.54830206  0.08771135 15.9204647
 46.96648794  0.996778    3.31427167  0.65882038 10.41733691]
  * std: [1.71313271e+00 1.81940308e-01 1.95404143e-01 1.42709225e+00
 4.71218617e-02 1.02685749e+01 3.30219278e+01 1.83946303e-03
 1.53911124e-01 1.72165005e-01 1.05927763e+00]

Test data statistics before normalization:
  * mean: [ 8.343125    0.5154375   0.27266667  2.51666667  0.08689583 15.76875
 45.30520833  0.99667367  3.30375     0.65658333 10.43614583]
  * std: [1.80263540e+00 1.71324851e-01 1.93172994e-01 1.36730930e+00
 4.68790285e-02 1.08825004e+01 3.25340058e+01 1.99070295e-03
 1.55079617e-01 1.62948079e-01 1.07920724e+00]

Training data statistics after normalization:
  * mean: [ 5.42907988e-16 -2.34148913e-16  1.65094827e-16 -6.66729109e-17
  3.41301806e-16  7.93725129e-18  1.58745026e-17 -4.02418641e-14
 -8.25474134e-16 -4.42898622e-16  1.17471319e-16]
  * std: [1. 1. 1. 1. 1. 1. 1.

## Problem 1: Generative Classifiers (Linear Discriminant Analysis)

Consider a classifier that predicts the class label $\widehat{y}$ for an observed feature vector $\mathbf{x}$ using the *maximum a posteriori* (MAP) classification rule:

$$
\widehat{y} = \arg\max_y p(y \mid \mathbf{x}).
$$

Assume the following:
1. The distribution of $\mathbf{x} \mid y$ is Gaussian.
2. The covariance matrix is shared by all the classes.

In the cell below, you have the skeleton of a `LDAClassifier` class that you will complete throughout this exercise to implement this classifier.


In [ ]:
import numpy as np
from scipy.stats import multivariate_normal

def pda_multivariate_normal(mean: np.ndarray, cov: np.ndarray, x: np.ndarray):
    # I thought that why shouldn't use multivariate normal and made this, and then saw 
    # the multivariate normal in the template, but i had already done this.
    F = mean.shape[0]
    temp = x - mean
    det_cov = np.linalg.det(cov)
    inv_cov = np.linalg.inv(cov)
    diagonal = np.einsum('ni,ij,nj->n', temp, inv_cov, temp)
    prefactor = 1 / np.sqrt((2 * np.pi)**F * det_cov)
    return prefactor * np.exp(-0.5 * diagonal)

class LDAClassifier:
    def fit(self, X: np.ndarray, y: np.ndarray) -> None:
        """
        Estimate the parameters of the classifier from the training data.

        Parameters:
        X: np.ndarray of shape (num_examples, num_features)
            The input data (features).
        y: np.ndarray of shape (num_examples,)
            The corresponding labels (target values).
        """
        # ToDo: Exercise 1.2
        # **Replace `pass` with your code**
        num_examples, num_features = X.shape
        self.classes = np.unique(y)
        num_classes = len(self.classes)

        self.means = np.zeros((num_classes, num_features))
        self.priors = np.zeros(num_classes)

        for idx, cls in enumerate(self.classes):
            X_cls = X[y == cls]
            self.means[idx] = X_cls.mean(axis=0)
            self.priors[idx] = X_cls.shape[0] / X.shape[0]

        cov_matrix = np.zeros((num_features, num_features))
        for idx, cls in enumerate(self.classes):
            X_cls = X[y == cls]
            cov_matrix += np.dot((X_cls - self.means[idx]).T, (X_cls - self.means[idx]))
        self.covariance = cov_matrix / (X.shape[0] - num_classes)

    def predict_proba(self, X: np.ndarray) -> np.ndarray:
        """
        Predict the class probabilities for the given input data.

        Parameters:
        X: np.ndarray of shape (num_examples, num_features)
            The input data (features).

        Returns:
        np.ndarray of shape (num_examples, num_classes)
            The predicted probabilities for each class.
        """
        # ToDo: Exercise 1.3
        # **Replace `pass` with your code**
        num_samples = X.shape[0]
        num_classes = len(self.classes)
        
        probs = np.zeros((num_samples, num_classes))
        


        for idx, cls in enumerate(self.classes):
            mean = self.means[idx]
            cov = self.covariance
            likelihood = pda_multivariate_normal(mean,cov, X)
            probs[:, idx] = likelihood * self.priors[idx]
        
        probs_sum = probs.sum(axis=1, keepdims=True)
        probs = probs / probs_sum
        return probs

    def predict(self, X: np.ndarray) -> np.ndarray:
        """
        Predict the class labels for the input data using the estimated model parameters.

        Parameters:
        X: np.ndarray of shape (num_examples, num_features)
            The input data (features) for which predictions are needed.

        Returns:
        np.ndarray of shape (num_examples,)
            The predicted class labels.
        """
        # ToDo: Exercise 1.4
        # **Replace `pass` with your code**
        probs = self.predict_proba(X)
        
        
        predictions = np.argmax(probs, axis=1)
        return self.classes[predictions]


### 1.1
Based on the assumptions given before, **identify all the parameters** in the model and **write down the expressions for their maximum likelihood estimates**. You do not need to show the derivation of the expressions, just the final result.

**Notation:**
- $N_i$: sample size in class $c_i$
- $N$: sample size
- $K$: number of classes


**Prior of class $c_i$**:

$
\hat{P}(c_i) = \frac{N_i}{N}
$

**Mean of class $c_i$**:

$
\hat{\mu}_i = \frac{1}{N_i - 1} \sum_{i:y_i = y}X_i
$

**Covariance Matrix of $c_i$**:

$
\hat{\sum}_i = \frac{1}{N_i - 1} \sum_{i = 1}^{N}(X_i - \hat{\mu}_{y_i})(X_i - \hat{\mu}_{y_i})^T
$

**Pooled covariance matrix $c_i$**:

$
\hat{\sum} = \frac{1}{N - K} \sum_{i = 1}^{K}(N_i - 1)\hat{\sum}_i
$

$
= \frac{1}{N - K} \sum_{i = 1}^{K}(X_i - \hat{\mu}_{y_i})(X_i - \hat{\mu}_{y_i})^T
$

### 1.2
In this exercise, you should write the code for the `fit` method of this classifier. This method should **estimate all model parameters** using the data provided as arguments. Specifically:

- `X`: A NumPy array of shape `(num_examples, num_features)` containing the training input examples $\mathbf{x}_1, \mathbf{x}_2, \dots, \mathbf{x}_n$.
- `y`: A NumPy array of shape `(num_examples,)` containing the corresponding labels $y_1, y_2, \dots, y_n$.

The model parameters should be saved as class attributes, so they can be used later during prediction.

After implementing the function, run the cell below to make sure that all the assertions pass and that the model parameter names and values are printed.


In [4]:
# instantiate the classifier and train it
lda_clf = LDAClassifier()
lda_clf.fit(X_train, y_train)

assert lda_clf.__dict__.keys(), ("`fit` method not implemented yet or not "
"creating the class attributes.")
print("Classifier attributes:")
for attr, value in lda_clf.__dict__.items():
    shape = f", shape {value.shape}" if hasattr(value, "shape") else ""
    print(f"  * {attr}{shape}: {value}")

Classifier attributes:
  * classes, shape (6,): [0 1 2 3 4 5]
  * means, shape (6, 11): [[ 1.37106912e-01  2.00725764e+00 -4.22060885e-01  1.25763224e-01
   8.43198752e-01 -4.25074483e-01 -6.24831922e-01  4.61367497e-01
   4.34272965e-01 -5.54625151e-01 -4.35939868e-01]
 [-2.69879785e-01  6.80509915e-01 -3.01228692e-01  5.76371290e-02
   1.74129734e-01 -2.46536238e-01 -2.00871210e-01 -1.57623531e-01
   2.66427180e-01 -2.57752328e-01 -2.01241142e-01]
 [-6.28732712e-02  2.80444275e-01 -1.28167508e-01  1.39499189e-02
   8.84356273e-02  1.03727345e-01  3.17583390e-01  2.08867168e-01
  -6.18446834e-02 -2.18073674e-01 -5.01012554e-01]
 [ 8.15345524e-04 -1.75140616e-01  5.84670637e-03 -6.61630507e-02
  -4.65407032e-02 -2.28261122e-02 -1.98174787e-01 -8.46028330e-02
   7.80116953e-02  1.06442906e-01  2.14973258e-01]
 [ 2.43136358e-01 -6.90115339e-01  4.72653602e-01  1.26863397e-01
  -2.32386401e-01 -1.77496745e-01 -3.57332901e-01 -4.13927324e-01
  -8.47228373e-02  4.78677468e-01  1.04271341e+0

### 1.3
Implement the `predict_proba` method of the `LDAClassifier` class. This method should use the model parameters estimated during the `fit` method to **predict the probabilities of each class** for new input data. Specifically, the input is:

- `X`: A NumPy array of shape `(num_examples, num_features)` containing the input examples $\mathbf{x}_1, \mathbf{x}_2, \dots, \mathbf{x}_n$ for which predictions are needed.

The method should return a NumPy array of shape `(num_examples, num_classes)` containing the predicted probabilities $p(y=0 \mid \mathbf{x}_i), p(y=1 \mid \mathbf{x}_i), \dots, p(y=C-1 \mid \mathbf{x}_i)$, for $i \in \{1, 2, \dots, n\}$.

After implementing the function, run the cell below to make sure that all the assertions pass.

In [5]:
# instantiate the classifier
# (we need to do this again because you probably changed the code of LDAClassifier
# after you ran the previous cell)
lda_clf = LDAClassifier()
lda_clf.fit(X_train, y_train)

# predict the class probabilities for the training data
probs = lda_clf.predict_proba(X_train)

assert probs is not None, ("`predict_proba` method not implemented yet or "
"not returning a valid array.")
assert probs.shape == (len(X_train), 6), ("`predict_proba` output should "
"have shape (num_examples, num_classes).")
assert np.all((probs >= 0) * (probs <= 1)), ("`predict_proba` output should be "
"probabilities.")
assert np.allclose(probs.sum(axis=1), 1), ("`predict_proba` output should be a "
"(proper) probability distribution.")
print("Your `predict_proba` looks good! 🚀")

Your `predict_proba` looks good! 🚀


### 1.4
Implement the `predict` method of the `LDAClassifier` class. This method should **predict the class labels** for new input data, by applying the MAP rule. Specifically, the input is:

- `X`: A NumPy array of shape `(num_examples, num_features)` containing the input examples $\mathbf{x}_1, \mathbf{x}_2, \dots, \mathbf{x}_n$ for which predictions are needed.

The method should return a NumPy array of shape `(num_examples,)` containing the predicted class labels $\widehat{y}_1, \widehat{y}_2, \dots, \widehat{y}_n$.

After implementing the function, run the cell below to make sure that all the assertions pass and that a summary of the model predictions is printed.


In [6]:
# instantiate the classifier
# (we need to do this again because you probably changed the code of LDAClassifier
# after you ran the previous cell)
lda_clf = LDAClassifier()
lda_clf.fit(X_train, y_train)

# predict the class labels for the training data
y_train_pred = lda_clf.predict(X_train)
assert y_train_pred is not None, ("`predict` method not implemented yet or not returning "
"any predictions.")
assert len(y_train_pred) == len(X_train), ("You should have one predicted "
"label for each input example.")
print("Your classifier predicted:")
for j in range(6):
    print(f"  * {(y_train_pred == j).sum()} training examples in class {j}.")
print()

# predict the class labels for the test data
y_test_pred = lda_clf.predict(X_test)
assert y_test_pred is not None, ("`predict` method not implemented yet or not returning "
"any predictions.")
assert len(y_test_pred) == len(X_test), ("You should have one predicted "
"label for each input example.")
print("Your classifier predicted:")
for j in range(6):
    print(f"  * {(y_test_pred == j).sum()} test examples in class {j}.")

Your classifier predicted:
  * 8 training examples in class 0.
  * 1 training examples in class 1.
  * 547 training examples in class 2.
  * 453 training examples in class 3.
  * 109 training examples in class 4.
  * 1 training examples in class 5.

Your classifier predicted:
  * 3 test examples in class 0.
  * 0 test examples in class 1.
  * 227 test examples in class 2.
  * 192 test examples in class 3.
  * 58 test examples in class 4.
  * 0 test examples in class 5.


### 1.5
Complete the code of the function `compute_accuracy`, which **computes the accuracy** of a given set of predictions when compared to the ground-truth labels. This function will be used to compute the training and test accuracies of our `LDAClassifier`.

After implementing the function, run the cell below to make sure that all the assertions pass and that the values of the training and test accuracies are reasonable.

In [7]:
def compute_accuracy(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    """
    Computes the accuracy of the classifier.

    Parameters:
    y_true: np.ndarray of shape (num_examples,)
        The true class labels.
    y_pred: np.ndarray of shape (num_examples,)
        The predicted class labels.

    Returns:
    float
        The accuracy of the classifier.
    """
    # ToDo:
    # **Replace `pass` with your code**
    num_correct = np.sum(y_true == y_pred)
    
    accuracy = num_correct / len(y_true)
    
    return accuracy


# compute the training accuracy
train_accuracy = compute_accuracy(y_train, y_train_pred)
assert train_accuracy is not None, ("`compute_accuracy` not implemented yet or "
"not returning a valid float.")
print(f"Training accuracy: {train_accuracy:.3f}")

# compute the test accuracy
test_accuracy = compute_accuracy(y_test, y_test_pred)
assert test_accuracy is not None, ("`compute_accuracy` not implemented yet or "
"not returning a valid float.")
print(f"Test accuracy: {test_accuracy:.3f}")


Training accuracy: 0.624
Test accuracy: 0.565


### 1.6
Explain why normalizing features to have zero mean and unit variance is an important preprocessing step **for this classifier**.

LDA assumes that the features within each category follow a Gaussian distribution and that all categories share the same covariance matrix. Normalizing the training data helps align the features to this assumption by standardizing their scales and variances, thereby reducing errors.

## Problem 2: Quadratic Discriminant Analysis (QDA)

You should now repeat the analysis performed on the previous exercise (items 1.2 to 1.5), but now using the QDA model. At the end, compare the performance of both methods and define which one would you choose to deploy. In the cell below, you have the skeleton of a `QDAClassifier` class that you will complete throughout this exercise to implement this classifier.

In [15]:
import numpy as np
from scipy.stats import multivariate_normal

class QDAClassifier:
    def fit(self, X: np.ndarray, y: np.ndarray) -> None:
        """
        Estimate the parameters of the classifier from the training data.

        Parameters:
        X: np.ndarray of shape (num_examples, num_features)
            The input data (features).
        y: np.ndarray of shape (num_examples,)
            The corresponding labels (target values).
        """
        # ToDo:
        num_examples, num_features = X.shape
        self.classes = np.unique(y)
        num_classes = len(self.classes)

        self.means = np.zeros((num_classes, num_features))
        self.priors = np.zeros(num_classes)

        for idx, cls in enumerate(self.classes):
            X_cls = X[y == cls]
            self.means[idx] = X_cls.mean(axis=0)
            self.priors[idx] = X_cls.shape[0] / X.shape[0]

        self.cov_matrix = np.zeros((num_features,num_features,num_classes))
        for idx, cls in enumerate(self.classes):
            X_cls = X[y == cls]
            self.cov_matrix[:,:,idx] = np.dot((X_cls - self.means[idx]).T, (X_cls - self.means[idx]))/(X_cls.shape[0] - 1)


    def predict_proba(self, X: np.ndarray) -> np.ndarray:
        """
        Predict the class probabilities for the given input data.

        Parameters:
        X: np.ndarray of shape (num_examples, num_features)
            The input data (features).

        Returns:
        np.ndarray of shape (num_examples, num_classes)
            The predicted probabilities for each class.
        """
        # ToDo:
        num_samples = X.shape[0]
        num_classes = len(self.classes)
        
        probs = np.zeros((num_samples, num_classes))

        for idx, cls in enumerate(self.classes):
            mean = self.means[idx]
            cov = self.cov_matrix[:,:,idx]
            print('X',X.shape)
            print('cov',cov.shape)
            print('mean',mean.shape)
            likelihood = multivariate_normal(X, mean,cov)
            probs[:, idx] = likelihood * self.priors[idx]
        
        probs_sum = probs.sum(axis=1, keepdims=True)
        print('probs',probs)
        probs = probs / probs_sum
        print('probs_sum',(probs_sum))
        print('probs',probs)
        return probs

    def predict(self, X: np.ndarray) -> np.ndarray:
        """
        Predict the class labels for the input data using the estimated model parameters.

        Parameters:
        X: np.ndarray of shape (num_examples, num_features)
            The input data (features) for which predictions are needed.

        Returns:
        np.ndarray of shape (num_examples,)
            The predicted class labels.
        """
        probs = self.predict_proba(X)
        
        
        predictions = np.argmax(probs, axis=1)
        return self.classes[predictions]




### 2.1

What is the difference between the estimated parameters of LDA and QDA?

*Your answer here*

### 2.2

Write your fit method:

In [13]:
# instantiate the classifier and train it
qda_clf = QDAClassifier()
qda_clf.fit(X_train, y_train)

assert qda_clf.__dict__.keys(), ("`fit` method not implemented yet or not "
"creating the class attributes.")
print("Classifier attributes:")
for attr, value in qda_clf.__dict__.items():
    shape = f", shape {value.shape}" if hasattr(value, "shape") else ""
    print(f"  * {attr}{shape}: {value}")

Classifier attributes:
  * classes, shape (6,): [0 1 2 3 4 5]
  * means, shape (6, 11): [[ 1.37106912e-01  2.00725764e+00 -4.22060885e-01  1.25763224e-01
   8.43198752e-01 -4.25074483e-01 -6.24831922e-01  4.61367497e-01
   4.34272965e-01 -5.54625151e-01 -4.35939868e-01]
 [-2.69879785e-01  6.80509915e-01 -3.01228692e-01  5.76371290e-02
   1.74129734e-01 -2.46536238e-01 -2.00871210e-01 -1.57623531e-01
   2.66427180e-01 -2.57752328e-01 -2.01241142e-01]
 [-6.28732712e-02  2.80444275e-01 -1.28167508e-01  1.39499189e-02
   8.84356273e-02  1.03727345e-01  3.17583390e-01  2.08867168e-01
  -6.18446834e-02 -2.18073674e-01 -5.01012554e-01]
 [ 8.15345524e-04 -1.75140616e-01  5.84670637e-03 -6.61630507e-02
  -4.65407032e-02 -2.28261122e-02 -1.98174787e-01 -8.46028330e-02
   7.80116953e-02  1.06442906e-01  2.14973258e-01]
 [ 2.43136358e-01 -6.90115339e-01  4.72653602e-01  1.26863397e-01
  -2.32386401e-01 -1.77496745e-01 -3.57332901e-01 -4.13927324e-01
  -8.47228373e-02  4.78677468e-01  1.04271341e+0

### 2.3

Implement the predict_proba method:

In [16]:
# instantiate the classifier
# (we need to do this again because you probably changed the code of QDAClassifier
# after you ran the previous cell)
qda_clf = QDAClassifier()
qda_clf.fit(X_train, y_train)

# predict the class probabilities for the training data
probs = qda_clf.predict_proba(X_train)

assert probs is not None, ("`predict_proba` method not implemented yet or "
"not returning a valid array.")
assert probs.shape == (len(X_train), 6), ("`predict_proba` output should "
"have shape (num_examples, num_classes).")
assert np.all((probs >= 0) * (probs <= 1)), ("`predict_proba` output should be "
"probabilities.")
assert np.allclose(probs.sum(axis=1), 1), ("`predict_proba` output should be a "
"(proper) probability distribution.")
print("Your `predict_proba` looks good! 🚀")

X (1119, 11)
cov (11, 11)
mean (11,)


ValueError: Array 'mean' must be a vector of length 12309.

### 2.4 

Write your predict method:

In [ ]:
# instantiate the classifier
# (we need to do this again because you probably changed the code of QDAClassifier
# after you ran the previous cell)
qda_clf = QDAClassifier()
qda_clf.fit(X_train, y_train)

# predict the class labels for the training data
y_train_pred = qda_clf.predict(X_train)
assert y_train_pred is not None, ("`predict` method not implemented yet or not returning "
"any predictions.")
assert len(y_train_pred) == len(X_train), ("You should have one predicted "
"label for each input example.")
print("Your classifier predicted:")
for j in range(6):
    print(f"  * {(y_train_pred == j).sum()} training examples in class {j}.")
print()

# predict the class labels for the test data
y_test_pred = qda_clf.predict(X_test)
assert y_test_pred is not None, ("`predict` method not implemented yet or not returning "
"any predictions.")
assert len(y_test_pred) == len(X_test), ("You should have one predicted "
"label for each input example.")
print("Your classifier predicted:")
for j in range(6):
    print(f"  * {(y_test_pred == j).sum()} test examples in class {j}.")

### 2.5
Using the compute_accuracy that you have already defined, compute the performance of the QDA on the given data. Compare the results with the ones obtained with LDA and define which one would you deploy.

In [ ]:
# compute the training accuracy
train_accuracy = compute_accuracy(y_train, y_train_pred)
assert train_accuracy is not None, ("`compute_accuracy` not implemented yet or "
"not returning a valid float.")
print(f"Training accuracy: {train_accuracy:.3f}")

# compute the test accuracy
test_accuracy = compute_accuracy(y_test, y_test_pred)
assert test_accuracy is not None, ("`compute_accuracy` not implemented yet or "
"not returning a valid float.")
print(f"Test accuracy: {test_accuracy:.3f}")

*Your answer here*